In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('All libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [4]:
CLIENT_ID = 'P5HBT4STVJQX2B3XR3MP2AV1ZDL5K52YK54M2C2HZ2H2HDIM' # your Foursquare ID
CLIENT_SECRET = 'ROBQSL43FMHWOSMMSUOH1EWDOWZRKPISRCP4ZYSVZX2AVPUN' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: P5HBT4STVJQX2B3XR3MP2AV1ZDL5K52YK54M2C2HZ2H2HDIM


In [17]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Toronto, ON', 'San Francisco, CA', 'Portland, OR', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") # COFFEE HOUSE CATEGORY ID
    results[city] = requests.get(url).json()

In [18]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

The Foursquare API Only gives us the nearest 100 venues in the city.

Let's first check out their densities by our eyes

In [19]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of coffee houses in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of coffee houses in New York, NY =  207
Showing Top 100
Total number of coffee houses in Toronto, ON =  229
Showing Top 100
Total number of coffee houses in San Francisco, CA =  210
Showing Top 100
Total number of coffee houses in Portland, OR =  193
Showing Top 100
Total number of coffee houses in Boston, MA =  199
Showing Top 100


In [20]:
maps[cities[0]]

In [21]:
maps[cities[1]]

In [22]:
maps[cities[2]]

In [23]:
maps[cities[3]]

In [24]:
maps[cities[4]]

We can see that San Francisco and Toronto are the cities with the highest number of coffee shops.  

However, the total number is not the same as density. Using the mean distances among coordinates will be used to assess how close the businesses are clustered. 

In [35]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.019933707590482175
Toronto, ON
Mean Distance from Mean coordinates
0.03580848539466422
San Francisco, CA
Mean Distance from Mean coordinates
0.026116104528479823
Portland, OR
Mean Distance from Mean coordinates
0.04096467122566089
Boston, MA
Mean Distance from Mean coordinates
0.02959977479772757


In [26]:
maps[cities[0]]

In [27]:
maps[cities[1]]

In [28]:
maps[cities[2]]

In [29]:
maps[cities[3]]

In [30]:
maps[cities[4]]

### Both Portland and Toronto have lower densities of coffee shops. This is interesting based upon the total number of shops being highest in Toronto. So in terms of opening a new shop, these cities may have locations that are "ripe" for possible outlets. However more research would be needed. 

Additional research may include: looking at density vs. area of city/bounds of total area, foot traffic, demographics, other business locations that may conducive to supplying customers, and ease of access/parking. It would be interesting to examine coffee shops in proximity to transit stations/subways due to increased foot traffic in these areas. 

In [44]:
city = 'Portland, OR'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-20]))# Ignore the ~10% biggest distance

Portland, OR
Mean Distance from Mean coordinates
0.027729364587055504


This analysis shows how density needs to be considered in addition to total number. With the furthest 20 shops removed, Portland, OR has what seems to be a density similar to other markets with high total numbers of shops.

#### This concludes the analysis of coffee shops by major cities for possible future sites. Toronto is a possible site due to lower density although it has a higher total number of shops. With further analysis, it seems like a suitable location could be identified. 